In [9]:
#importing the required library
import string
import requests
import pandas as pd
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import openpyxl
from nltk.corpus import stopwords
import snscrape.modules.twitter as sntwitter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
query = "#BorderGavaskarTrophy2023"      #searching tweets with this Hashtag
tweets = []                              #creating an empty list to store tweets data
limit = 5000                             #limiting the tweets search to 5000 tweets


for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content])
        
df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet'])   #saving tweet data in a pandas dataframe
df

<ipython-input-14-d0da5bd7fa80>:12: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content])
<ipython-input-14-d0da5bd7fa80>:12: FutureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.username, tweet.content])


,Date,User,Tweet
0,2023-02-11 13:36:45+00:00,jumbo__22,#BorderGavaskarTrophy2023\n😆😆 https://t.co/jo9...
1,2023-02-11 13:35:40+00:00,Harshkhunteta18,And they say it's Australia's best chance to w...
2,2023-02-11 13:35:15+00:00,RanganiParas,Shami scored 37 runs on that turning pitch and...
3,2023-02-11 13:34:28+00:00,ArneshDe,Boys negotiating room rates for 14th February....
4,2023-02-11 13:33:10+00:00,ArnavHot,Father-son same problem\n\nMunna Bhai didn't w...
...,...,...,...
4995,2023-02-10 07:29:06+00:00,Munnaaaaahhhhh,What a brilliant hundred from the Hitman. A ca...
4996,2023-02-10 07:28:58+00:00,parthj67,Century in all formats as a player.\nCentury i...
4997,2023-02-10 07:28:52+00:00,harrykaj23,Rohit Sharma scores his 9th test century! Not ...
4998,2023-02-10 07:28:47+00:00,dhonimsddd,Rohith Sharma well deserved century🔥🔥🔥9th test...


In [13]:
#using openpyxl library to create and update an excel sheet to load the analysis data

excel = openpyxl.Workbook() #creating a workbook
sheet = excel.active        #going to active sheet
sheet.title= 'Output_Data'  #naming the sheet
sheet.append(['Tweet', 'Polarity Score','Profanity Score'])  #adding required columns to the sheet

for i in range(len(df)):

  tweet=df['Tweet'][i]
  # Get the list of stop words in English
  stop_words = nltk.corpus.stopwords.words('english')
  sentences = nltk.sent_tokenize(tweet)    #tokenizing text file by each sentence of tweet
  tokens = nltk.word_tokenize(tweet)       #tokenizing text file by each word for each tweet
  total_words = []
  for w in tokens:
    if w not in stop_words and w not in string.punctuation:
       total_words.append(w)
  word_count = len(total_words)
   # calculating positive score, negative score, polarity and subjectivity scores

  with open('negative-words.txt', 'r', encoding='latin-1') as neg_word:
     n_words = neg_word.read()    # loading negative words
             

  with open('positive-words.txt', 'r', encoding='latin-1') as pos_word:    #loading positive words
     p_words = pos_word.read()

         
  
  positive_words = nltk.word_tokenize(p_words)
  negative_words = nltk.word_tokenize(n_words)
  negative_score = 0
  positive_score = 0

  for word in total_words:
    if word in negative_words:
      negative_score +=1
    elif word in positive_words:
      positive_score +=1
        
    # polarity score
  polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
  Profanity_score = (negative_score/len(total_words))
  sheet.append([tweet,polarity_score,Profanity_score])
  

excel.save('Output_Data.xlsx')                #saving the excel file 